## Importing Libraries

In [ ]:
!pip install https://github.com/ElsevierDev/elsapy/archive/master.zip
#Install necessary extensions/libraries

In [ ]:
#xml.etree.ElementTree — The ElementTree XML API
import xml.etree.ElementTree as ET

In [ ]:
#Essential libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Essential libraries to make API requests with ElsSearch
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json
import csv
import pprint
import requests
import xmltodict
import urllib3
import re

## Science Direct Full Text API

In [ ]:
#Load the doi file previously extracted from ElsSearch API
#Our data is stored in <path>
df = pd.read_csv(r'C:\Users\swath\doi_6000.csv')
df

### Institute Token for subscription

In [ ]:
#Append "apikey" and "insttoken" as suggest in the ElsSearch document into a config file
config = {
    "apikey": "2dc442325fc67f2f275ec3157ef8df65",
 "insttoken": "6beb1f6c29d85f50029bf11c8de94d1b"
    }

client = ElsClient(config['apikey'])
client.inst_token = config['insttoken']

In [ ]:
#Create string1 and string2 to join doi with institoken to make a single URL
string1 = "https://api.elsevier.com/content/article/doi/" 
string2 = "?apiKey=2dc442325fc67f2f275ec3157ef8df65&insttoken=6beb1f6c29d85f50029bf11c8de94d1b"

#Access every DOI in the previous file and append the new URL to another column
df['Link'] = df['DOI'].apply(lambda x: string1 + str(x) + string2)

In [ ]:
#Example of a Link for 1st DOI in the file
df['Link'][0]

In [ ]:
#Save the combined abstracts in another CSV file
df.to_csv(r'C:\Users\swath\OneDrive\Desktop\Projects\NLP_Coal\doi_6000_withlink.csv', index=False)

### Loading the file

In [ ]:
df_new = pd.read_csv(r'C:\Users\swath\OneDrive\Desktop\Projects\NLP_Coal\doi_6000_withlink.csv')
df_new.head()

## Check

A checkloop to extract the number of full-text papers returned based on previous API request

In [ ]:
#Define a header's dictionary to pass through requests
headers_dict = {"X-ELS-APIKey": "2dc442325fc67f2f275ec3157ef8df65", "X-ELS-Insttoken": "6beb1f6c29d85f50029bf11c8de94d1b", "Accept": "application/xml"}

#Indicators for every file
yes = 0
no = 0

#Loop to extract every file from the list of unique (5945) DOIs
for i in range(0, 5945):
    
    #x takes response of the HTTP request, passes link
    x = requests.get(df_new['Link'][i], headers=headers_dict)
    #print(x.text) #check
        
    #Save response as XML file (stored in root path of the user)
    with open("full_text.xml", 'wb') as f:
        f.write(x.content)
    
    #tree calls the XML file stored at root location
    tree = ET.parse(r'C:\Users\swath\full_text.xml')
    root = tree.getroot()
    
    original_text = root.find('{http://www.elsevier.com/xml/svapi/article/dtd}originalText')
    #print(originaltext)
    doc = original_text.find('{http://www.elsevier.com/xml/xocs/dtd}doc')
    #print(doc)
    serial_item = doc.find('{http://www.elsevier.com/xml/xocs/dtd}serial-item')
    #print(serial_item)

    if serial_item != None:
        yes +=1
        print(yes)
    else:
        #print("Full text for this paper doesn't exist")
        no+=1
        
print(yes)
print(no)

## XML Parsing

In [ ]:
#Define a header's dictionary to pass through requests
headers_dict = {"X-ELS-APIKey": "2dc442325fc67f2f275ec3157ef8df65", "X-ELS-Insttoken": "6beb1f6c29d85f50029bf11c8de94d1b", "Accept": "application/xml"}

#x takes response of the HTTP request, passes link
x = requests.get(df_new['Link'][15], headers=headers_dict)
#print(x.text) #check
    
#Save response as XML file (stored in root path of the user)
with open("full_text.xml", 'wb') as f:
    f.write(x.content)

In [ ]:
df_new['Link'][15]

In [ ]:
#Reading the data

#<?xml version="1.0" encoding="UTF-8"?>

#tree calls the XML file stored at root location
tree = ET.parse(r'C:\Users\swath\full_text.xml')
root = tree.getroot()
    

In [ ]:
#The XML file Looks like this for DOI: 10.1016/j.tet.2021.132473

# <full-text-retrieval-response xmlns="http://www.elsevier.com/xml/svapi/article/dtd" 
#    xmlns:bk="http://www.elsevier.com/xml/bk/dtd" xmlns:cals="http://www.elsevier.com/xml/common/cals/dtd" 
#    xmlns:ce="http://www.elsevier.com/xml/common/dtd" 
#    xmlns:ja="http://www.elsevier.com/xml/ja/dtd" 
#    xmlns:mml="http://www.w3.org/1998/Math/MathML" 
#    xmlns:sa="http://www.elsevier.com/xml/common/struct-aff/dtd" 
#    xmlns:sb="http://www.elsevier.com/xml/common/struct-bib/dtd" 
#    xmlns:tb="http://www.elsevier.com/xml/common/table/dtd" 
#    xmlns:xlink="http://www.w3.org/1999/xlink" 
#    xmlns:xocs="http://www.elsevier.com/xml/xocs/dtd" 
#    xmlns:dc="http://purl.org/dc/elements/1.1/" 
#    xmlns:dcterms="http://purl.org/dc/terms/" 
#    xmlns:prism="http://prismstandard.org/namespaces/basic/2.0/" 
#    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
# <coredata>
# ... #This part has all basic info 
# </coredata>
# <objects>
# ... #This part is unnecessary
# </objects>
# <scopus-id>85116092071</scopus-id>
# <scopus-eid>2-s2.0-85116092071</scopus-eid>
# <link href="https://api.elsevier.com/content/abstract/scopus_id/85116092071" rel="abstract"/>
# <originalText>
# ... #This is the main part
      #head
            #title
            #author
            #abstract
            #keywords
      #body 
            #sections
            #para
                #subsections
                    #para
                
      #tail (useless)
# </originalText>
# </full-text-retrieval-response>


### Code to Extract Coredata

In [ ]:
#Coredata is present in every paper

#Element.findall() finds only elements with a tag which are direct children of the current element
#Element.find() finds the first child with a particular tag

#Manually entered XML with Namespaces
#df = pd.DataFrame({'url': url, 'doi': doi, 'title': title, 'pub_name': pub_name, 'type': type_, 'abstract': description})

for entry in root.findall('{http://www.elsevier.com/xml/svapi/article/dtd}coredata'):
    
    url = []; doi =[]; title = []; pub_name =[]; type_ = []
    description = []
    #CHECK CODE
    url.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text)
    doi.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi').text)
    title.append(entry.find('{http://purl.org/dc/elements/1.1/}title').text)
    pub_name.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text)
    type_.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}aggregationType').text)
    description.append(entry.find('{http://purl.org/dc/elements/1.1/}description').text)
    description[0] = " ".join(description[0].split())
    print(url, doi, title, pub_name, type_, description)
    print('\n')


### Code to extract head

In [ ]:
original_text = root.find('{http://www.elsevier.com/xml/svapi/article/dtd}originalText')
#print(originaltext)
doc = original_text.find('{http://www.elsevier.com/xml/xocs/dtd}doc')
#print(doc)
serial_item = doc.find('{http://www.elsevier.com/xml/xocs/dtd}serial-item')
#print(serial_item)

keyword_list = []
author_list = []
if serial_item != None:
    article = serial_item.find('{http://www.elsevier.com/xml/ja/dtd}article')
    #print(article)
    head = article.find('{http://www.elsevier.com/xml/ja/dtd}head')
    #print(head)
    author_group = head.find('{http://www.elsevier.com/xml/common/dtd}author-group')
    #print(author_group)

    keywords_ = head.find('{http://www.elsevier.com/xml/common/dtd}keywords')
    #print(keywords_)

    for author in author_group.findall('{http://www.elsevier.com/xml/common/dtd}author'):
        name = author.find('{http://www.elsevier.com/xml/common/dtd}given-name').text
        surname = author.find('{http://www.elsevier.com/xml/common/dtd}surname').text
        author_list.append(name + ' ' + surname)
        print('\t')

    for word in keywords_.itertext():
        keyword_list.append(word)

    keyword_list = "".join(keyword_list)
    keyword_list = list(keyword_list.split())

    print(keyword_list)
    print(author_list)

    
else:
    print("Full text for this paper doesn't exist")

d = dict(url = np.array(url), 
         doi = np.array(doi), 
         title = np.array(title), 
         pub_name = np.array(pub_name), 
         Type = np.array(type_), 
         abstract= np.array(description), 
         authors= np.array(author_list), 
         keyword =np.array(keyword_list))

#Save it in a dataframe
df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in d.items() ]))
df.fillna('', inplace=True)
df

### Code to extract body

In [ ]:
body = article.find('{http://www.elsevier.com/xml/ja/dtd}body')
#print(body) #check
sections = body.find('{http://www.elsevier.com/xml/common/dtd}sections')
#print(sections) #check

#lists for dataframe
label_list = [] 
title_list = []
para_list = []

#A multiple iterative loop to iterate through every section, subsection, paragraphs and sentences
#Same logic of root to next node to root to subsequent node is followed at every loop

for section1 in sections.findall('{http://www.elsevier.com/xml/common/dtd}section'):
    if section1.find('{http://www.elsevier.com/xml/common/dtd}label') != None:
        label = section1.find('{http://www.elsevier.com/xml/common/dtd}label').text
        label_list.append(label)
        print(label)

        section_title_list = []
        section_title = section1.find('{http://www.elsevier.com/xml/common/dtd}section-title')
        for sec in section_title.itertext():
            section_title_list.append(sec)

        section_title_list = "".join(section_title_list)
        section_title_list = list(section_title_list.split()) 
        section_title_list = " ".join(section_title_list)
        title_list.append(section_title_list)
        print(section_title_list)

        total_para = []
        
        #Code to extract main text before entering the subsection
        #paragraph = section1.find('{http://www.elsevier.com/xml/common/dtd}para')
        for paragraph in section1.findall('{http://www.elsevier.com/xml/common/dtd}para'):
            if paragraph != None:
                #paragraph = paragraph.text
                #print(paragraph)
                #print('\n')
                paragraph1 = []
                for p in paragraph.itertext():
                    paragraph1.append(p)

                paragraph1 = "".join(paragraph1)
                paragraph1 = list(paragraph1.split()) 
                paragraph1 = " ".join(paragraph1)
                total_para.append(paragraph1)

        total_para = " ".join(total_para)
        print(total_para)
        para_list.append(total_para)
        print('\n')
        
        #Code to extract substection paragraphs
        for section2 in section1.findall('{http://www.elsevier.com/xml/common/dtd}section'):
            if section2.find('{http://www.elsevier.com/xml/common/dtd}label') !=None:
                label = section2.find('{http://www.elsevier.com/xml/common/dtd}label').text
                label_list.append(label)
                print('sub-section ', label)

                sub_section_title_list = []

                sub_section_title = section2.find('{http://www.elsevier.com/xml/common/dtd}section-title')
                for sub in sub_section_title.itertext():
                    sub_section_title_list.append(sub)

                sub_section_title_list = "".join(sub_section_title_list)
                sub_section_title_list = list(sub_section_title_list.split()) 
                sub_section_title_list = " ".join(sub_section_title_list)
                title_list.append(sub_section_title_list)
                #print('sub-section-title: ', sub_section_title_list)

                #Code to extract multiple paragraphs under same subsection
                para1 = []
                for para in section2.itertext():
                #findall('{http://www.elsevier.com/xml/common/dtd}para'):
                    #paragraph = para.
                    #for i in paragraph: 
                    para1.append(para)

                para1 = "".join(para1)
                para1 = list(para1.split()) 
                para1 = " ".join(para1)
                print(para1) 
                para_list.append(para1)
                print('\n')

In [ ]:
#Create a dictionary to store all the values
d = dict(url = np.array(url), 
         doi = np.array(doi), 
         title = np.array(title), 
         pub_name = np.array(pub_name), 
         Type = np.array(type_), 
         abstract= np.array(description), 
         authors= np.array(author_list), 
         keyword =np.array(keyword_list),
         title_label = np.array(label_list),
         titles = np.array(title_list),
         text = np.array(para_list))

df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in d.items() ]))
df.fillna('', inplace=True)
df

In [ ]:
df.to_csv('C://Users//swath//Downloads//paper_xml_parsed.csv')

## Saving the text to CSV

In [ ]:
info_dict = {} #This will create a dictionary where we will store information about the searches

In [ ]:
for entry in root.findall('{http://www.elsevier.com/xml/svapi/article/dtd}coredata'):
    
    url = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
    doi = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi').text
    title = entry.find('{http://purl.org/dc/elements/1.1/}title').text
    pub_name = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
    type_ = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}aggregationType').text
    description = entry.find('{http://purl.org/dc/elements/1.1/}description').text
    print(url, doi, title, pub_name, type_, description)
    print('\n')
    
    #CHECK CODE
    info_dict['URL'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
    info_dict['DOI'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi').text
    info_dict['Title'] = entry.find('{http://purl.org/dc/elements/1.1/}title').text
    info_dict['Pub_Name'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
    info_dict['Type'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}aggregationType').text
    info_dict['Abstract'] = entry.find('{http://purl.org/dc/elements/1.1/}description').text
    print(url, doi, title, pub_name, type_, description)
    print('\n')

In [ ]:
original_text = root.find('{http://www.elsevier.com/xml/svapi/article/dtd}originalText')
#print(originaltext)
doc = original_text.find('{http://www.elsevier.com/xml/xocs/dtd}doc')
#print(doc)
serial_item = doc.find('{http://www.elsevier.com/xml/xocs/dtd}serial-item')
#print(serial_item)
article = serial_item.find('{http://www.elsevier.com/xml/ja/dtd}article')
#print(article)

head = article.find('{http://www.elsevier.com/xml/ja/dtd}head')
#print(head)
author_group = head.find('{http://www.elsevier.com/xml/common/dtd}author-group')
#print(author_group)

keywords_ = head.find('{http://www.elsevier.com/xml/common/dtd}keywords')
#print(keywords_)
full_name_list = []
full_name_dict = {}
for author in author_group.findall('{http://www.elsevier.com/xml/common/dtd}author'):
    name = author.find('{http://www.elsevier.com/xml/common/dtd}given-name').text
    surname = author.find('{http://www.elsevier.com/xml/common/dtd}surname').text
    full_name_dict['Name'] = name + surname
    full_name_list.append(full_name_dict)
    
keyword_list = []
keyword_dict = {}

for keyword in keywords_.findall('{http://www.elsevier.com/xml/common/dtd}keyword'):
    keyword = keyword.find('{http://www.elsevier.com/xml/common/dtd}text').text
    keyword_dict['Keywords'] = keyword
    keyword_list.append(keyword_dict)
    print(keyword_list)

print(full_name_list)


The above code is run in loop until all doi are exhaisted and stored in CSV files